# LSTM models on the UCI datasets
- LSTM Models are slow to train. The newly proposed TCNs are shown to be simpler and easier to train, along with better performance

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [1]:
from keras.utils import to_categorical

# Data loading utilities

In [3]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [4]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [16]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Permute, Reshape
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [17]:
# LSTM Model
num_hidden_lstm = 16
win_len = 128
dim = 9
num_classes = 6

model = Sequential()
model.add(LSTM(num_hidden_lstm, 
               input_shape=(win_len,dim), 
               return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(num_hidden_lstm, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128, 16)           1664      
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 16)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 102       
Total params: 3,878
Trainable params: 3,878
Non-trainable params: 0
_________________________________________________________________


In [14]:
batch_size = 32
epochs = 10
verbose = 1
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/10
7352/7352 [==============================] - 14s 2ms/step - loss: 0.6375 - accuracy: 0.7293
Epoch 2/10
7352/7352 [==============================] - 14s 2ms/step - loss: 0.6926 - accuracy: 0.7168
Epoch 3/10
7352/7352 [==============================] - 14s 2ms/step - loss: 0.5473 - accuracy: 0.7637
Epoch 4/10
7352/7352 [==============================] - 15s 2ms/step - loss: 0.5087 - accuracy: 0.7758
Epoch 5/10
7352/7352 [==============================] - 17s 2ms/step - loss: 0.4211 - accuracy: 0.8207
Epoch 6/10
7352/7352 [==============================] - 24s 3ms/step - loss: 0.3796 - accuracy: 0.8531
Epoch 7/10
7352/7352 [==============================] - 35s 5ms/step - loss: 0.2945 - accuracy: 0.9045
Epoch 8/10
7352/7352 [==============================] - 30s 4ms/step - loss: 0.2670 - accuracy: 0.9124
Epoch 9/10
7352/7352 [==============================] - 25s 3ms/step - loss: 0.2104 - accuracy: 0.9339
Epoch 10/10
7352/7352 [==============================] - 30s 4ms/step - l

In [15]:
# evaluate model
_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.8720732927322388


# A Simple MLP to classify this

In [27]:
num_hidden_lstm = 16
win_len = 128
dim = 9
num_classes = 6

model = Sequential()
model.add(Dense(64, input_shape=(win_len*dim,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 64)                73792     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 198       
Total params: 77,126
Trainable params: 77,126
Non-trainable params: 0
__________________________________________________

In [28]:
batch_size = 32
epochs = 100
verbose = 1
model.fit(trainX.reshape(-1,win_len*dim ), trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/100
7352/7352 [==============================] - 1s 76us/step - loss: 0.9610 - accuracy: 0.6109
Epoch 2/100
7352/7352 [==============================] - 0s 43us/step - loss: 0.4925 - accuracy: 0.8116
Epoch 3/100
7352/7352 [==============================] - 0s 42us/step - loss: 0.3670 - accuracy: 0.8644
Epoch 4/100
7352/7352 [==============================] - 0s 48us/step - loss: 0.2923 - accuracy: 0.8898
Epoch 5/100
7352/7352 [==============================] - 0s 52us/step - loss: 0.2649 - accuracy: 0.8981
Epoch 6/100
7352/7352 [==============================] - 0s 48us/step - loss: 0.2384 - accuracy: 0.9094
Epoch 7/100
7352/7352 [==============================] - 0s 43us/step - loss: 0.2253 - accuracy: 0.9119
Epoch 8/100
7352/7352 [==============================] - 0s 52us/step - loss: 0.2191 - accuracy: 0.9144
Epoch 9/100
7352/7352 [==============================] - 0s 49us/step - loss: 0.1937 - accuracy: 0.9267
Epoch 10/100
7352/7352 [==============================] - 0s 46u

7352/7352 [==============================] - 0s 47us/step - loss: 0.1130 - accuracy: 0.9538
Epoch 80/100
7352/7352 [==============================] - 0s 46us/step - loss: 0.1046 - accuracy: 0.9585
Epoch 81/100
7352/7352 [==============================] - 0s 46us/step - loss: 0.0977 - accuracy: 0.9595
Epoch 82/100
7352/7352 [==============================] - 0s 60us/step - loss: 0.1030 - accuracy: 0.9574
Epoch 83/100
7352/7352 [==============================] - 0s 58us/step - loss: 0.1041 - accuracy: 0.9566
Epoch 84/100
7352/7352 [==============================] - 0s 64us/step - loss: 0.1030 - accuracy: 0.9573
Epoch 85/100
7352/7352 [==============================] - 0s 59us/step - loss: 0.1076 - accuracy: 0.9565
Epoch 86/100
7352/7352 [==============================] - 0s 53us/step - loss: 0.0989 - accuracy: 0.9584
Epoch 87/100
7352/7352 [==============================] - 0s 55us/step - loss: 0.0939 - accuracy: 0.9595
Epoch 88/100
7352/7352 [==============================] - 0s 52us/st

In [29]:
# evaluate model
_, accuracy = model.evaluate(testX.reshape(-1,win_len*dim ), testy, batch_size=batch_size, verbose=0)

print('Test accuracy of model is:', accuracy)

Test accuracy of model is: 0.8819137811660767
